# MTG Card Data Exploration and ML/AI Analysis Template

This notebook serves as a template for exploring Magic: The Gathering card data from Cosmos DB and performing deep ML/AI analysis.

## Table of Contents
1. [Data Loading](#Data-Loading)
2. [Data Exploration](#Data-Exploration)
3. [Data Preprocessing](#Data-Preprocessing)
4. [Feature Engineering](#Feature-Engineering)
5. [Model Development](#Model-Development)
6. [Model Evaluation](#Model-Evaluation)
7. [Results and Insights](#Results-and-Insights)

In [1]:
# Data Loading Section
## Import Required Libraries

import os
import sys
import pandas as pd
import numpy as np
from pymongo import MongoClient
from pymongo.errors import PyMongoError

# ML/AI Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add project root to path for importing local modules
sys.path.append('../')
from data_engine.cosmos_driver import get_mongo_client, get_collection

In [3]:
# Load Card Data from Cosmos DB

def load_card_data():
    """
    Load all card data from Cosmos DB into a pandas DataFrame.
    """
    try:
        # Connect to Cosmos DB
        client = get_mongo_client()

        # Get database and collection names from environment or defaults
        database_name = os.environ.get('COSMOS_DB_NAME', 'cards')
        container_name = os.environ.get('COSMOS_CONTAINER_NAME', 'mtgecorec')

        # Get collection
        collection = get_collection(client, container_name, database_name)

        # Fetch all documents
        cards = list(collection.find())

        # Convert to DataFrame
        df = pd.DataFrame(cards)

        # Remove MongoDB _id column if present
        if '_id' in df.columns:
            df = df.drop('_id', axis=1)

        print(f"Successfully loaded {len(df)} cards from Cosmos DB")
        print(f"DataFrame shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")

        return df

    except Exception as e:
        print(f"Error loading data from Cosmos DB: {e}")
        return None
    finally:
        if 'client' in locals():
            client.close()

# Load the data
df_cards = load_card_data()

# Display first few rows
if df_cards is not None:
    display(df_cards.head())
else:
    print("Failed to load card data. Please check your Cosmos DB connection and environment variables.")

Successfully loaded 109820 cards from Cosmos DB
DataFrame shape: (109820, 88)
Columns: ['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors', 'color_identity', 'keywords', 'produced_mana', 'legalities', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'prices', 'related_uris', 'purchase_uris', 'day_uploaded', 'mtgo_foil_id', 'power', 'toughness', 'flavor_text', 'edhrec_rank', 'penny_rank', 'all_parts', 'promo

,object,id,oracle_id,multiverse_ids,mtgo_id,arena_id,tcgplayer_id,cardmarket_id,name,lang,...,flavor_name,attraction_lights,color_indicator,printed_type_line,printed_text,variation_of,life_modifier,hand_modifier,content_warning,defense
0,card,0000419b-0bba-4488-8f7a-6194544ce91e,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,[668564],129825.0,91829.0,558404.0,777725.0,Forest,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,NaN,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,NaN,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,66119.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Exploration

In this section, we'll explore the loaded card data to understand its structure, distributions, and key characteristics.

In [ ]:
# Basic Data Exploration

if df_cards is not None:
    print("DataFrame Info:")
    print(df_cards.info())

    print("\n" + "="*50)
    print("Descriptive Statistics:")
    print(df_cards.describe())

    print("\n" + "="*50)
    print("Missing Values:")
    print(df_cards.isnull().sum())

    print("\n" + "="*50)
    print("Unique Values in Categorical Columns:")
    categorical_cols = df_cards.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        try:
            unique_count = df_cards[col].nunique()
            print(f"{col}: {unique_count} unique values")
            if unique_count <= 10:
                print(f"  Values: {df_cards[col].unique()}")
        except TypeError:
            # Handle unhashable types like lists/dicts
            print(f"{col}: Contains complex data types (lists/dicts), {len(df_cards)} total values")
else:
    print("No data available for exploration.")

## Data Preprocessing

Clean and prepare the data for analysis. Handle missing values, encode categorical variables, normalize numerical features, etc.

```python
# Example preprocessing steps (uncomment and modify as needed)

# # Handle missing values
# df_cards = df_cards.dropna()  # or use imputation strategies

# # Encode categorical variables
# le = LabelEncoder()
# df_cards['rarity_encoded'] = le.fit_transform(df_cards['rarity'])

# # Normalize numerical features
# scaler = StandardScaler()
# numerical_cols = ['price']  # add other numerical columns
# df_cards[numerical_cols] = scaler.fit_transform(df_cards[numerical_cols])
```

## Feature Engineering

Create new features from existing data that might be useful for ML models.

```python
# Example feature engineering (uncomment and modify as needed)

# # Extract features from text fields
# df_cards['name_length'] = df_cards['name'].str.len()
# df_cards['has_flavor_text'] = df_cards['flavor_text'].notna()

# # Create color-based features
# df_cards['color_count'] = df_cards['colors'].apply(lambda x: len(x) if isinstance(x, list) else 0)
# df_cards['is_multicolored'] = df_cards['color_count'] > 1

# # Date features
# df_cards['release_year'] = pd.to_datetime(df_cards['release_date']).dt.year
```

## Model Development

Build and train ML models for various tasks such as price prediction, rarity classification, etc.

```python
# Example model development (uncomment and modify as needed)

# # Prepare features and target
# features = ['rarity_encoded', 'color_count', 'release_year']  # add relevant features
# target = 'price'  # or another target variable

# X = df_cards[features]
# y = df_cards[target]

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train model
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)
```

## Model Evaluation

Evaluate model performance using appropriate metrics.

```python
# Example evaluation (uncomment and modify as needed)

# # Classification report
# print(classification_report(y_test, y_pred))

# # Confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.show()

# # Feature importance
# feature_importance = pd.DataFrame({
#     'feature': features,
#     'importance': model.feature_importances_
# }).sort_values('importance', ascending=False)

# plt.figure(figsize=(10, 6))
# sns.barplot(x='importance', y='feature', data=feature_importance)
# plt.title('Feature Importance')
# plt.show()
```

## Results and Insights

Summarize findings, insights, and potential next steps.

- Key findings from data exploration
- Model performance summary
- Business insights
- Recommendations for future analysis

## Next Steps

- [ ] Fine-tune model hyperparameters
- [ ] Try different algorithms (XGBoost, Neural Networks, etc.)
- [ ] Perform cross-validation
- [ ] Deploy model for predictions
- [ ] Set up monitoring and retraining pipeline

In [2]:
# Convert notebook to HTML for web display
import sys
sys.path.append('../scripts')
from convert_notebook import convert_notebook_to_html

# Run the conversion
convert_notebook_to_html()

Notebook converted to HTML and saved to static/card_explore.html
